# Fine-Tuning for QA

In [1]:
import os
import torch
import time
import numpy as np 
import pandas as pd
import tensorflow as tf
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration, GenerationConfig, TrainingArguments, Trainer

In [2]:
import datasets
import transformers
print(datasets.__version__)
print(transformers.__version__)

2.16.1
4.36.2


Define config parameters

In [3]:
# model_name='google/flan-t5-small'
model_name='t5-small'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'  

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = base_model.to(device)

/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/huggingface_hub-0.24.0-py3.8.egg/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.


Load datasets

In [5]:
dataset_bitext_train = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[:16000]")
dataset_bitext_test = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[-4000:-2000]")
dataset_bitext_validation = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train[-2000:]")

In [5]:
dataset_local_train = load_dataset('csv', split='train[:80%]', data_files={'local_merged.csv'})
dataset_local_train

Generating train split: 0 examples [00:00, ? examples/s]

/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 10
})

In [6]:
dataset_local_test = load_dataset('csv', split='train[-20%:-10%]', data_files={'local_merged.csv'})
dataset_local_test

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 1
})

In [7]:
dataset_local_validation = load_dataset('csv', split='train[-10%:]', data_files={'local_merged.csv'})
dataset_local_validation

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 1
})

In [8]:
dataset_train_merged = concatenate_datasets(
    [
        dataset_bitext_train, 
        dataset_local_train
    ]
)
dataset_test_merged = concatenate_datasets(
    [
        dataset_bitext_test, 
        dataset_local_test
    ]
)
dataset_validation_merged = concatenate_datasets(
    [
        dataset_bitext_validation,
        dataset_local_validation
    ]
)

In [9]:
dataset_train_merged.to_csv('train_merged.csv', index=False)
dataset_test_merged.to_csv('test_merged.csv', index=False)
dataset_validation_merged.to_csv('validation_merged.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1109899

In [6]:
dataset = load_dataset('csv', data_files={
    "train": "train_merged.csv", 
    "test": "test_merged.csv", 
    "validation": "validation_merged.csv"
    })

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 16010
    })
    test: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 2001
    })
    validation: Dataset({
        features: ['flags', 'instruction', 'category', 'intent', 'response'],
        num_rows: 2001
    })
})

In [8]:
dataset['train'][0]

{'flags': 'B',
 'instruction': 'question about cancelling order {{Order Number}}',
 'category': 'ORDER',
 'intent': 'cancel_order',
 'response': "I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you."}

Explore dataset

In [9]:
dataset_train = pd.read_csv('./train_merged.csv')

In [10]:
dataset_train.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token="<|im_end|>")

In [18]:
# CHAT_ML_TEMPLATE = """
# {% for message in messages %}
#     {% if message['role'] == 'user' %}
#         {{'<|im_start|>user\n' + message['content'].strip() + '<|im_end|>' }}
#     {% elif message['role'] == 'system' %}
#         {{'<|im_start|>system\n' + message['content'].strip() + '<|im_end|>' }}
#     {% elif message['role'] == 'assistant' %}
#         {{'<|im_start|>assistant\n'  + message['content'] + '<|im_end|>' }}
#     {% endif %}
# {% endfor %}
# """
# tokenizer.chat_template = CHAT_ML_TEMPLATE

In [12]:
tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [10]:
# CHAT_ML_TEMPLATE = """ 
# {%- for message in messages %}
#     {%- if message['role'] == 'user' %}
#         {{- bos_token + '[INST] ' + message['content'].strip() + ' [/INST]' }}
#     {%- elif message['role'] == 'system' %}
#         {{- '<<SYS>>\\n' + message['content'].strip() + '\\n<</SYS>>\\n\\n' }}
#     {%- elif message['role'] == 'assistant' %}
#         {{- '[ASST] '  + message['content'] + ' [/ASST]' + eos_token }}
#     {%- endif %}
# {%- endfor %}
# """

# template = tokenizer.chat_template
# template = template.replace("SYS", "SYSTEM")  # Change the system token
# tokenizer.chat_template = template  # Set the new template
# tokenizer.push_to_hub("model_name")  # Upload your new template to the Hub!

In [13]:
chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

# tokenizer.use_default_system_prompt = False
tokenizer.apply_chat_template(chat, tokenize=False)

"<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing great. How can I help you today?<|im_end|>\n<|im_start|>user\nI'd like to show off how chat templating works!<|im_end|>\n"

In [14]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = base_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
<pad> Wie alt sind Sie?</s>


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [15]:
def get_prompt(data, index):
    instruction = data.iloc[index]['instruction']
    response = data.iloc[index]['response']
    
    prompt = f'''
    instruction:
    {instruction}
    
    response:
    {response}
    
    '''
    
    return prompt

def get_instruction(data,index):
    instruction = data.iloc[index]['instruction']
    return f'''
    
    instruction:
    {instruction}
    
    response:
    
    '''

In [16]:
test_prompt = get_prompt(dataset_train,0) + get_instruction(dataset_train,10)
print(test_prompt)


    instruction:
    question about cancelling order {{Order Number}}
    
    response:
    I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.
    
    
    
    instruction:
    i dont know what to do to cancel order {{Order Number}}
    
    response:
    
    


In [17]:
input_text = tokenizer.encode(test_prompt,return_tensors="pt")
output = tokenizer.decode(base_model.generate(input_text)[0])
print(output)

<pad><extra_id_0>: I have understood you have a question regarding cancelling order <unk>Order Number


In [18]:
idx = 100
idx_var = 50

check = dataset_train.iloc[idx]['instruction']
check2 = dataset_train.iloc[idx]['response']

test = dataset_train.iloc[idx_var]['instruction']
test_response = dataset_train.iloc[idx_var]['response']

print(f'instruction:\n{test}\n\nresponse:\n')

print(f'======================\nexpected reponse: \n {test_response}')

instruction:
canceling order {{Order Number}}

response:

expected reponse: 
 I appreciate that you're seeking assistance with canceling your order tied to the order number {{Order Number}}, and I'm here to guide you through the cancellation process. Please follow these steps:

1. Access Your Account: Log in to your {{Online Company Portal Info}} using your username and password.
2. Navigate to Your Orders: Once you're logged in, locate the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.
3. Locate the Order: Look for the specific order associated with the order number {{Order Number}}.
4. Initiate the Cancellation: Within the order details, you should find an option to '{{Online Order Interaction}}'. Click on it to start the cancellation process.
5. Confirm the Cancellation: The system may ask for your confirmation or gather feedback. Please provide the necessary information accurately.

If you encounter any difficulties or have further questions, our dedicate

In [19]:
def tokenize_function(example):
    start_prompt = 'Instruction:\n'
    end_prompt = '\nResponse:'
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["instruction"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["response"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

In [20]:
shuffled_dataset = dataset.shuffle(seed=42)

In [21]:
tokenized_datasets = shuffled_dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['flags', 'instruction', 'category', 'intent', 'response'])

Map:   0%|          | 0/16010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 16010
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2001
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2001
    })
})

In [23]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
finetuned_model = finetuned_model.to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [24]:
output_dir = 'training-t5'

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=5e-3,
    num_train_epochs=2,
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',        # evaluation strategy to adopt during training
    eval_steps=500,                 
)

trainer = Trainer(
    model=finetuned_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer.evaluate()

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 1.732622742652893,
 'eval_model_preparation_time': 0.0019,
 'eval_runtime': 78.5127,
 'eval_samples_per_second': 25.486,
 'eval_steps_per_second': 1.605}

In [26]:
%%time

trainer.train()

  0%|          | 0/2002 [00:00<?, ?it/s]

{'loss': 1.0416, 'grad_norm': 0.12597836554050446, 'learning_rate': 0.004875124875124875, 'epoch': 0.05}
{'loss': 0.4333, 'grad_norm': 0.09609200805425644, 'learning_rate': 0.00475024975024975, 'epoch': 0.1}
{'loss': 0.3822, 'grad_norm': 0.0963071659207344, 'learning_rate': 0.004625374625374625, 'epoch': 0.15}
{'loss': 0.3633, 'grad_norm': 0.12200659513473511, 'learning_rate': 0.004500499500499501, 'epoch': 0.2}
{'loss': 0.3547, 'grad_norm': 0.10900302231311798, 'learning_rate': 0.004375624375624376, 'epoch': 0.25}
{'loss': 0.3325, 'grad_norm': 0.13998627662658691, 'learning_rate': 0.004250749250749251, 'epoch': 0.3}
{'loss': 0.3285, 'grad_norm': 0.22718411684036255, 'learning_rate': 0.004125874125874126, 'epoch': 0.35}
{'loss': 0.3063, 'grad_norm': 0.12131989002227783, 'learning_rate': 0.004000999000999001, 'epoch': 0.4}
{'loss': 0.2987, 'grad_norm': 0.11856796592473984, 'learning_rate': 0.003876123876123876, 'epoch': 0.45}
{'loss': 0.2952, 'grad_norm': 0.0806303396821022, 'learning_r

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 0.31155863404273987, 'eval_model_preparation_time': 0.0019, 'eval_runtime': 76.8942, 'eval_samples_per_second': 26.023, 'eval_steps_per_second': 1.639, 'epoch': 0.5}
{'loss': 0.2889, 'grad_norm': 0.08306342363357544, 'learning_rate': 0.003626373626373626, 'epoch': 0.55}
{'loss': 0.2761, 'grad_norm': 0.07841289043426514, 'learning_rate': 0.0035014985014985016, 'epoch': 0.6}
{'loss': 0.267, 'grad_norm': 0.09067802131175995, 'learning_rate': 0.0033766233766233766, 'epoch': 0.65}
{'loss': 0.2741, 'grad_norm': 0.10285599529743195, 'learning_rate': 0.0032517482517482516, 'epoch': 0.7}
{'loss': 0.2633, 'grad_norm': 0.08768812566995621, 'learning_rate': 0.003126873126873127, 'epoch': 0.75}
{'loss': 0.2688, 'grad_norm': 0.076161690056324, 'learning_rate': 0.003001998001998002, 'epoch': 0.8}
{'loss': 0.2602, 'grad_norm': 0.10809271782636642, 'learning_rate': 0.0028771228771228776, 'epoch': 0.85}
{'loss': 0.2584, 'grad_norm': 0.09937774389982224, 'learning_rate': 0.00275224775224775

  0%|          | 0/126 [00:00<?, ?it/s]

...

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 0.2774794399738312, 'eval_model_preparation_time': 0.0019, 'eval_runtime': 77.862, 'eval_samples_per_second': 25.699, 'eval_steps_per_second': 1.618, 'epoch': 1.5}
{'loss': 0.2156, 'grad_norm': 0.06207076460123062, 'learning_rate': 0.0011288711288711289, 'epoch': 1.55}
{'loss': 0.2277, 'grad_norm': 0.08566132932901382, 'learning_rate': 0.001003996003996004, 'epoch': 1.6}
{'loss': 0.2222, 'grad_norm': 0.08713541179895401, 'learning_rate': 0.0008791208791208792, 'epoch': 1.65}
{'loss': 0.2143, 'grad_norm': 0.07805190235376358, 'learning_rate': 0.0007542457542457542, 'epoch': 1.7}
{'loss': 0.2195, 'grad_norm': 0.07622843235731125, 'learning_rate': 0.0006293706293706295, 'epoch': 1.75}
{'loss': 0.2125, 'grad_norm': 0.06771530956029892, 'learning_rate': 0.0005044955044955045, 'epoch': 1.8}
{'loss': 0.2134, 'grad_norm': 0.08639439940452576, 'learning_rate': 0.0003796203796203796, 'epoch': 1.85}
{'loss': 0.217, 'grad_norm': 0.06294690817594528, 'learning_rate': 0.000254745254745

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 0.2703797519207001, 'eval_model_preparation_time': 0.0019, 'eval_runtime': 77.3108, 'eval_samples_per_second': 25.883, 'eval_steps_per_second': 1.63, 'epoch': 2.0}
{'train_runtime': 21531.7692, 'train_samples_per_second': 1.487, 'train_steps_per_second': 0.093, 'train_loss': 0.2819580896780803, 'epoch': 2.0}
CPU times: user 19min 17s, sys: 55min 14s, total: 1h 14min 32s
Wall time: 5h 58min 51s


TrainOutput(global_step=2002, training_loss=0.2819580896780803, metrics={'train_runtime': 21531.7692, 'train_samples_per_second': 1.487, 'train_steps_per_second': 0.093, 'total_flos': 4333644483133440.0, 'train_loss': 0.2819580896780803, 'epoch': 2.0})

In [27]:
finetuned_model.save_pretrained("model-t5")

In [28]:
tokenizer.save_pretrained("model-t5")

('model-t5/tokenizer_config.json',
 'model-t5/special_tokens_map.json',
 'model-t5/tokenizer.json')

In [29]:
model_path = "model-t5"
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
finetuned_model = finetuned_model.to('cpu')

finetuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [30]:
def process_data_sample(example):
    
    processed_example = "<|system|>\n You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.</s>\n<|user|>\n" + example["instruction"] + "</s>\n<|response|>\n"

    return processed_example

In [31]:
def get_response(question):
    input_str = process_data_sample(
        {
            "instruction": question,
        }
    )
    inputs = finetuned_tokenizer(input_str, return_tensors="pt").to(device)
    outputs = finetuned_model.generate(
        **inputs,
        do_sample=True,
        top_k=1,
        temperature=0.1,
        max_new_tokens=256,
        pad_token_id=finetuned_tokenizer.eos_token_id
        )
    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [32]:
question = "i have a question about cancelling order {{Order Number}}"
get_response(question)

"I've realized you have a question regarding canceling order Order Number. I apologize for any inconvenience this may have caused you. I apologize for any inconvenience this may have caused you. I apologize for any inconvenience this may have caused you. I apologize for any inconvenience this may have caused you. I understand that you have a question regarding canceling order Order Number and I'm here to assist you. Please feel free to ask, and I'll be more than happy to assist you. Thank you for bringing this matter to our attention, and I'm unable to cancel the order. I'm unable to assist you with canceling the order and I apologize for any inconvenience caused. Thank you for bringing this matter to our attention, and I apologize for any inconvenience caused. Thank you for bringing this matter to our attention, and I apologize for any inconvenience caused. Thank you for bringing this matter to our attention, and I'm here to assist you. Thank you for bringing this matter to our attent

In [33]:
for i in range(0,10, 1):
    print('Instruction: ' + dataset['test'][i]['instruction'])
    print('Predict. :' + get_response(dataset['test'][i]['instruction']))
    print('Expected: ' + dataset['test'][i]['response'])
    print('=================================\n')

Instruction: do ya have an e-mail to send a review for a service
Predict. :I hear you! I'm following you that you're requesting assistance in a chatbot. I'm here to help you with that. You can connect with a live chat agent, and you're chatting with a live agent. We're here to help you. Please feel free to connect with a live agent, and I'll be happy to connect you with a live agent. We're always here to help you.
Expected: Indeed! I'm thrilled to assist you with sending a review for our service. While we don't have an email specifically dedicated to receiving reviews, we have alternative methods that you can utilize to provide your feedback. One option is to visit our website and navigate to the "Contact Us" or "Feedback" page. There, you can fill out a form or send a message directly to our customer support team. Another option is to share your review on our social media platforms or review websites. We highly value your feedback and look forward to hearing about your experience! Is 

In [28]:
# messages = [
#     {
#         "role": "system",
#         "content": "You are a friendly chatbot who always responds in the style of a pirate",
#     },
#     {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
#  ]
# tokenized_chat = finetuned_tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
# print(finetuned_tokenizer.decode(tokenized_chat[0]))

PEFT

In [50]:
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from safetensors.torch import save_model

In [37]:
peft_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
peft_model = peft_model.to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [41]:
output_dir = 'training-t5-peft'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    learning_rate=1e-3,
    num_train_epochs= 3
)

peft_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
    
peft_trainer = SFTTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['test'],
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True,
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/trl/trainer/sft_tra

In [42]:
peft_trainer.evaluate()

  0%|          | 0/21 [00:00<?, ?it/s]

{'eval_loss': 43.771480560302734,
 'eval_model_preparation_time': 0.0078,
 'eval_runtime': 5.1103,
 'eval_samples_per_second': 19.764,
 'eval_steps_per_second': 4.109}

In [43]:
%%time
peft_trainer.train()

  0%|          | 0/486 [00:00<?, ?it/s]

{'train_runtime': 368.2763, 'train_samples_per_second': 6.598, 'train_steps_per_second': 1.32, 'train_loss': 2.973497932339892, 'epoch': 3.0}
CPU times: user 4min 33s, sys: 25.1 s, total: 4min 58s
Wall time: 6min 8s


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /google/flan-t5-small/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f8bc14e6670>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 64745307-67d0-49c9-a005-6d3a3628461e)') - silently ignoring the lookup for the file config.json in google/flan-t5-small.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in google/flan-t5-small - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=486, training_loss=2.973497932339892, metrics={'train_runtime': 368.2763, 'train_samples_per_second': 6.598, 'train_steps_per_second': 1.32, 'total_flos': 461987622420480.0, 'train_loss': 2.973497932339892, 'epoch': 3.0})

In [44]:
peft_model.save_pretrained("model-peft")
tokenizer.save_pretrained("model-peft")

('model-peft/tokenizer_config.json',
 'model-peft/special_tokens_map.json',
 'model-peft/tokenizer.json')

In [45]:
save_model(peft_model, "peft_model.safetensors")

In [46]:
peft_model = get_peft_model(base_model, peft_config)
print(peft_model.print_trainable_parameters())

trainable params: 1,376,256 || all params: 78,337,408 || trainable%: 1.7568
None


In [48]:
peft_model_path = "./model-peft"
peft_model = AutoModelForSeq2SeqLM.from_pretrained(peft_model_path)
peft_model = peft_model.to('cpu')

peft_tokenizer = AutoTokenizer.from_pretrained(peft_model_path)

Some weights of the model checkpoint at ./model-peft were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.0.SelfAttention.q.base_layer.weight', 'decoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.base_layer.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.base_layer.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_A.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.base_layer.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_A.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_B.default.weight', 'decoder.block.1.layer.0.SelfAttention.q.base_layer.weight', 'decoder.block.1.layer.0.SelfAttention.q.lora_A.d

In [52]:
test_prompt = f'instruction:\n{test}\n\nresponse:\n'
input_text = peft_tokenizer(test_prompt,return_tensors="pt").input_ids
output = peft_model.generate(
    input_ids=input_text, 
    generation_config=GenerationConfig(max_new_tokens=200, 
    num_beams=1))

model_text_output = peft_tokenizer.decode(output[0], skip_special_tokens=True)

print(model_text_output)

ship ship the ship ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship ship the ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the ship the ship ship the 


In [53]:
chat1 = [
    {"role": "user", "content": "Which is bigger, the moon or the sun?"},
    {"role": "assistant", "content": "The sun."}
]
chat2 = [
    {"role": "user", "content": "Which is bigger, a virus or a bacterium?"},
    {"role": "assistant", "content": "A bacterium."}
]

In [ ]:
datasetx = Dataset.from_dict({"chat": [chat1, chat2]})
datasetx = datasetx.map(lambda x: {"formatted_chat": peft_tokenizer.apply_chat_template(x["chat"], tokenize=False, add_generation_prompt=False)})
print(datasetx['formatted_chat'][0])

Evaluate

In [34]:
import evaluate

In [35]:
rouge = evaluate.load('rouge')

In [36]:
instructions = dataset['test'][0:10]['instruction']
response = dataset['test'][0:10]['response']

base_model_responses = []
finetuned_model_responses = []
peft_model_responses = []

for instruction in instructions:
    prompt = f""" 
    instruction:
    {instruction}

    response:
    """
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    base_model_outputs = base_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    base_model_output = tokenizer.decode(base_model_outputs[0], skip_special_tokens=True)

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    finetuned_model_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)


    # peft_model_outputs = peft_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    # peft_model_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    base_model_responses.append(base_model_output)
    finetuned_model_responses.append(finetuned_model_output)
    # peft_model_responses.append(peft_model_output)

In [37]:
zipped_responses = list(zip(response, base_model_responses, finetuned_model_responses))
df = pd.DataFrame(zipped_responses, columns=['human','base','finetuned'])
df

,human,base,finetuned
0,Indeed! I'm thrilled to assist you with sendin...,instruction: do ya have an e-mail to send a re...,I hear you! I'm following you that you're look...
1,We greatly appreciate your willingness to prov...,response: where could I send feedback for your...,I'm on it! I'm here to guide you on where to f...
2,Thanks for dropping us a line to us for assist...,instruction: a,I'm happy to help! I'm here to provide you wit...
3,Thank you for reaching out! I completely under...,instruction: i have got to write an opinion fo...,I'm happy to help! I'm here to assist you in w...
4,I'm on it! I'm here to assist you in submittin...,instruction: i,I'm happy to help! I'm here to provide you wit...
5,Thank you for expressing your desire to provid...,I would like to leave a comment for your compa...,I'm sorry to hear that you would like to leave...
6,Thank you for your interest in providing feedb...,instruction instruction instruction instructio...,I'm on it! I'm here to guide you through where...
7,Of course! I'm here to guide you through the p...,help me submit my feedback about your company ...,I'm happy to help! I'm here to provide you wit...
8,Assuredly! We are here to guide you through th...,: me a product? response:,I'm happy to help! I'm here to provide you wit...
9,We're here for you and expressing your interes...,instruction: assistance to send some feedback ...,We're here for you! I'm aware that you need as...


In [38]:
response = dataset['test'][0:10]['response']

base_model_results = rouge.compute(
    predictions=base_model_responses,
    references=response,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_model_responses,
    references=response,
    use_aggregator=True,
    use_stemmer=True,
)

# peft_model_results = rouge.compute(
#     predictions=peft_model_responses,
#     references=response,
#     use_aggregator=True,
#     use_stemmer=True,
# )

In [39]:
print('Base\n',base_model_results)
print('Fine-tuned\n',finetuned_model_results)
# print('\PEFT\n',peft_model_results)

Base
 {'rouge1': 0.0988895164143127, 'rouge2': 0.028177263597074384, 'rougeL': 0.06916503448497743, 'rougeLsum': 0.07003667617767625}
Fine-tuned
 {'rouge1': 0.4213982629168878, 'rouge2': 0.13223336639484112, 'rougeL': 0.26287176767024556, 'rougeLsum': 0.26311347560765597}
